<a href="https://colab.research.google.com/github/inesboukhizzou/EEG-classifier/blob/main/eeg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%autosave 60

Autosaving every 60 seconds


In [ ]:
!pip install mne pyriemann

In [ ]:
import mne

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# loading the data
from mne.datasets import eegbci

runs = [6,10,14]
f_names = []
for subject in range(1,2):
  f_names += eegbci.load_data(subject, runs)

In [ ]:
from mne import concatenate_raws
from mne.io import read_raw_edf

raw = concatenate_raws([read_raw_edf(f, preload=True) for f in f_names]) # load the data

# remove dots from channel names
raw.rename_channels(lambda x: x.strip("."))

eegbci.standardize(raw) # standardize the data
montage = mne.channels.make_standard_montage('standard_1005')
raw.set_montage(montage)

Extracting EDF parameters from /root/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R06.edf...
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /root/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R10.edf...
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Extracting EDF parameters from /root/mne_data/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R14.edf...
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...


<RawEDF | S001R06.edf, 64 x 60000 (375.0 s), ~29.4 MiB, data loaded>

In [ ]:
print(raw.info)

<Info | 9 non-empty values
 bads: []
 ch_names: FC5, FC3, FC1, FCz, FC2, FC4, FC6, C5, C3, C1, Cz, C2, C4, C6, ...
 chs: 64 EEG
 custom_ref_applied: False
 dig: 67 items (3 Cardinal, 64 EEG)
 highpass: 0.0 Hz
 lowpass: 80.0 Hz
 meas_date: 2009-08-12 16:15:00 UTC
 nchan: 64
 projs: []
 sfreq: 160.0 Hz
 subject_info: <subject_info | his_id: X, sex: 0, last_name: X>
>


In [ ]:
raw.filter(7., 35., fir_design='firwin')

Filtering raw data in 3 contiguous segments
Setting up band-pass filter from 7 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 7.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 265 samples (1.656 s)



<RawEDF | S001R06.edf, 64 x 60000 (375.0 s), ~29.4 MiB, data loaded>

In [ ]:
from mne import pick_types

picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, exclude='bads')

picks = picks[::2] # we pick odd numbered channels

In [ ]:
from mne import Epochs, events_from_annotations

tmin, tmax = 1., 2.

event_id = dict(hands=2, feet=3)
events, _ = events_from_annotations(raw, event_id=dict(T1=2, T2=3))

epochs = Epochs(raw, events, event_id, tmin, tmax, baseline=None, preload=True)

Used Annotations descriptions: [np.str_('T1'), np.str_('T2')]
Not setting metadata
45 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 45 events and 161 original time points ...
0 bad epochs dropped


In [ ]:
labels = epochs.events[:, -1] -2

In [ ]:
# split data into K folds

from sklearn.model_selection import KFold

kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [ ]:
epochs_data = 1e6 * epochs.get_data() # converting the data from volts to micro volts

In [ ]:
# Linear Discriminant Analysis

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from mne.decoding import CSP

lda = LinearDiscriminantAnalysis()

csp = CSP(n_components=4, reg=None, log=True, norm_trace=False)

clf = Pipeline([('CSP', csp), ('LDA', lda)])
scores = cross_val_score(clf, epochs_data, labels, cv=kf, n_jobs=1)

Computing rank from data with rank=None
    Using tolerance 1.6e+02 (2.2e-16 eps * 64 dim * 1.1e+16  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 1.5e+02 (2.2e-16 eps * 64 dim * 1.1e+16  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=0 covariance using EMPIRICAL
Done.
Estimating class=1 covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 1.6e+02 (2.2e-16 eps * 64 dim * 1.1e+16  max singular value)
    Estimated rank (data): 64
    data: rank 64 computed from 64 data channels with 0 projectors
Reducing data rank from 64 -> 64
Estimating class=0 covariance using EMP

In [ ]:
# results
class_balance = np.mean(labels == labels[0])
class_balance = max(class_balance, 1. - class_balance)

print("CSP + LDA classification accuracy:%f / Chance level: %f" % (np.mean(scores), class_balance))

CSP + LDA classification accuracy:0.905000 / Chance level: 0.533333


In [ ]:
# Riemannian Classification
from pyriemann.estimation import Covariances

cov_data = Covariances().transform(epochs_data) # transforming each epoch into a covariance matrix

In [ ]:
# Classification by Minimum Distance to Mean.

from pyriemann.classification import MDM

mdm = MDM(metric=dict(mean='riemann', distance='riemann'))
scores = cross_val_score(mdm, cov_data, labels, cv=kf, n_jobs=1)

In [ ]:
class_balance = np.mean(labels == labels[0])
class_balance = max(class_balance, 1. - class_balance)

print("MDM Classification accuracy: %f / Chance level: %f" % (np.mean(scores), class_balance))

MDM Classification accuracy: 0.790000 / Chance level: 0.533333


In [ ]:
from pyriemann.classification import TSClassifier
tsc = TSClassifier()
scores = cross_val_score(tsc, cov_data, labels, cv=kf, n_jobs=1)

In [ ]:
class_balance = np.mean(labels == labels[0])
class_balance = max(class_balance, 1. - class_balance)

print("TSC Classification accuracy: %f / Chance level: %f" % (np.mean(scores), class_balance))

TSC Classification accuracy: 0.810000 / Chance level: 0.533333
